In [ ]:
import numpy as np
import pandas as pd
import math as m
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf
# from scipy import *
import scipy.linalg
from scipy.optimize import minimize
from scipy.linalg import eig, eigh
from scipy.fft import fft

from sklearn.datasets import make_swiss_roll
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph, NearestNeighbors
from sklearn.decomposition import KernelPCA, PCA
from sklearn.manifold import TSNE
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors

#import scaleogram as scg
import matplotlib.pyplot as plt
import warnings

from visuals import *
from my_lib import *
from SSA_lib import SSA

In [ ]:
warnings.simplefilter('ignore')

# plt.rcParams['text.usetex'] = True
plt.rcParams['figure.figsize'] = 5, 5
plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 8
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['axes.labelsize'] = 8

In [ ]:
def CartesianToSpherical(point):
    r = np.sqrt(sum(point ** 2))
    n = len(point)
    phi = np.zeros(n - 1)
    
    for i in range(n - 2):
        phi[i] = np.arccos(point[i] / np.sqrt(sum(point[i:] ** 2)))
        
    if point[-1] >= 0:
        phi[n - 2] = np.arccos(point[n - 2] / np.sqrt(point[n - 1] ** 2 + point[n - 2] ** 2))
    else:
        phi[n - 2] = 2 * np.pi - np.arccos(point[n - 2] / np.sqrt(point[n - 1] ** 2 + point[n - 2] ** 2))
        
    return np.hstack((phi, r))


def TrajectoryToSpherical(tr):
    tr_spherical = np.zeros(tr.shape)
    for i, point in enumerate(tr):
        tr_spherical[i] = CartesianToSpherical(point)
    return tr_spherical


def SphericalToCartesian(point):
    phi, r = point[:-1], point[-1]
    n = len(point)
    x = np.zeros(n) 
    cur = r
    
    for i in range(n - 1):
        x[i] = cur * np.cos(phi[i])
        cur *= np.sin(phi[i])

    x[n - 1] = cur
    return x
    

def TrajectoryToCartesian(tr):
    tr_cartesian = np.zeros(tr.shape)
    for i, point in enumerate(tr):
        tr_cartesian[i] = SphericalToCartesian(point)
    return tr_cartesia

def HankelMatrix(X, L):  
    N = X.shape[0]
    return scipy.linalg.hankel(X[ : N - L + 1], X[N - L : N])

In [ ]:
dt = 450 * 20
data = pd.read_csv('data/long_walk_100_acc.csv', delimiter =';', decimal=',')[7007-470*2:7007+dt]

frequency = len(data)/(data['time'].values[-1]-data['time'].values[0])

assert 490 < frequency < 510, f'Bad frequency {frequency}'
print(f'Frequency {frequency}')

x_acc = ( (data['X_value'].values)**2 + (data['Y_value'].values)**2 + (data['Z_value'].values)**2)**.5
_m = np.mean(x_acc)
_std = (sum((x_acc - _m)**2)/len(x_acc))**.5

x_acc = (x_acc-_m)

t = (data['time'].values).astype(float).reshape([-1,])
t = np.linspace(0,t[-1]-t[0],len(x_acc))

fig = go.Figure()
fig.add_scatter(y = x_acc, mode='lines', name='Sum squares')
fig.show()

In [ ]:
X = HankelMatrix(x_acc,750)

pca_ = PCA(n_components=3)
X_PCA = pca_.fit_transform(X)

fig_2 = go.Figure()
fig_2.update_layout(autosize=False, width=1000, height=1000)

fig_2.add_trace(go.Scatter3d(x=X_PCA[:,0],
                             y=X_PCA[:,1],
                             z=X_PCA[:,2],
                             mode='lines'
                            )
                )


fig_2.layout.template = 'plotly_white'
fig_2.show()

In [ ]:
X_PCA_sp = TrajectoryToSpherical(X_PCA)

In [ ]:
# fig_2 = go.Figure()
# fig_2.update_layout(autosize=False, width=1000, height=1000)

# fig_2.add_trace(go.Scatter3d(x=X_PCA_sp[:,0],
#                              y=X_PCA_sp[:,1],
#                              z=X_PCA_sp[:,2],
#                              mode='markers',
#                              marker=dict(size=3)
#                             )
#                 )


# fig_2.layout.template = 'plotly_white'
# fig_2.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.special import sph_harm

In [ ]:
N = 400
phi = np.linspace(0, np.pi, N)
theta = np.linspace(0, 2*np.pi, N)
phi, theta = np.meshgrid(phi, theta)

# The Cartesian coordinates of the unit sphere
x = np.sin(phi) * np.cos(theta)
y = np.sin(phi) * np.sin(theta)
z = np.cos(phi)

traj_matrix = np.zeros_like(phi)

Coor_init = np.concatenate((phi.reshape((len(theta)**2,1)),theta.reshape((len(phi)**2,1))),axis=1)

nbrs = NearestNeighbors(n_neighbors = 30, p = 1)
nbrs.fit(Coor_init)
_, indices = nbrs.kneighbors(X_PCA_sp[:,(0,1)])

clmn = (indices%N).reshape(-1)
row = (indices//N).reshape(-1)
traj_matrix[row,clmn] = 1

In [ ]:
fig = go.Figure()
fig.add_trace(go.Surface(x=x,
                         y=y,
                         z=z,
                         surfacecolor = traj_matrix))
fig.show()

In [ ]:
class sph_harm_approximation():
    
    def __init__(self, L_harm = 10, N = 100):
        self.L = L_harm
        self.N = N
        self.ind = 0
        
    def _makemodel_(self):
        phi = np.linspace(0, np.pi, self.N)
        theta = np.linspace(0, 2 * np.pi, self.N)
        self.phi, self.theta = np.meshgrid(phi, theta)
        model = []
        
        for l in range(self.L):
            for m in range(l+1):
                model.append(sph_harm(m, l, self.theta, self.phi).real)
                
        model = np.array(model)
        temp_map = model.sum(axis = 0)
        tmax, tmin = temp_map.max(), temp_map.min()
        model = (model - tmin)/(tmax - tmin)
        return model
    
    def fit(self, train_map):
        self.init_map = self._makemodel_()
        self.train_map = train_map
        return self
    
    def loss(self, w):
        temp_map = self.train_map.copy()*0
        
        for i in range(int(self.L*(self.L+1)/2)):
            temp_map += w[i] *  self.init_map[i]
        
        _loss = np.sum((temp_map - self.train_map)**2)
        
        if self.ind%1000 == 0:
            print(f'Iter num:{self.ind}, total loss:{_loss}')
        self.ind += 1
        return _loss
    
    def div(self, w):
        _div = np.zeros_like(w)
        
        temp_map = self.train_map.copy()*0
        
        for i in range(int(self.L*(self.L+1)/2)):
            temp_map += w[i] *  self.init_map[i]

        for i in range(len(w)):
            _div[i] = np.sum(2 * (temp_map - self.train_map) * self.init_map[i])
        
        return _div
    
    def set_iternum(self, iter_num = 0):
        self.ind = iter_num
    
    def predict_all(self, w):
        
        temp_map = self.train_map.copy()*0
        
        for i in range(int(self.L*(self.L+1)/2)):
            temp_map += w[i] *  self.init_map[i]
            
        tmax, tmin = temp_map.max(), temp_map.min()
        temp_map = (temp_map - tmin)/(tmax - tmin)
            
        return temp_map

In [ ]:
n = 60
Model = sph_harm_approximation(L_harm = n, N = N)
Model.fit(traj_matrix)

In [ ]:
w0 = np.ones(int(n*(n+1)/2))

new_map = Model.predict_all(w0)
fig = go.Figure()
fig.add_trace(go.Surface(x=x,
                         y=y,
                         z=z,
                         surfacecolor = new_map))

fig.add_trace(go.Surface(x=x,
                         y=y,
                         z=z,
                         surfacecolor = traj_matrix))
fig.show()

In [ ]:
Model.set_iternum()
res = minimize(Model.loss, w0,
               method = 'BFGS',
               jac = Model.div,
               options={'maxfev': 100,
                        'maxiter': 100,
                        'disp': True,
                        'xtol': 5,
                        'ftol': 5
                       }
              )
Model.set_iternum()

In [ ]:
new_map = Model.predict_all(res.x)
# A = new_map * (new_map > 0).astype(float)# + traj_matrix

# amax, amin = A.max(), A.min()
# A = (A - amin)/(amax - amin)

fig = go.Figure()
fig.add_trace(go.Surface(x=x,
                         y=y,
                         z=z,
                         surfacecolor = new_map))
fig.show()

